# Carregar os módulos necessários para a análise

In [1]:
import os
import numpy as np
import pandas as pd

import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
import xyzservices.providers as xyz

print(os)
print(np)
print(pd)
print(folium)
print(MarkerCluster)
print(HeatMap)
print(xyz)

<module 'os' from '/usr/lib/python3.10/os.py'>
<module 'numpy' from '/home/chico_linux/.local/lib/python3.10/site-packages/numpy/__init__.py'>
<module 'pandas' from '/home/chico_linux/.local/lib/python3.10/site-packages/pandas/__init__.py'>
<module 'folium' from '/home/chico_linux/.local/lib/python3.10/site-packages/folium/__init__.py'>
<class 'folium.plugins.marker_cluster.MarkerCluster'>
<class 'folium.plugins.heat_map.HeatMap'>
{'OpenStreetMap': {'Mapnik': {'url': 'https://tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'html_attribution': '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors', 'attribution': '(C) OpenStreetMap contributors', 'name': 'OpenStreetMap.Mapnik'}, 'DE': {'url': 'https://tile.openstreetmap.de/{z}/{x}/{y}.png', 'max_zoom': 18, 'html_attribution': '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors', 'attribution': '(C) OpenStreetMap contributors', 'name': 'OpenStreetMap.DE'}, '

# Carregar as bases necessárias

In [2]:
parques = os.listdir('src/data')
parques.remove('CADMUN.csv')

new_born = pd.DataFrame()

for i in parques:
    path = 'src/data/' + i
    dn = pd.read_parquet(path)
    new_born = pd.concat([new_born, dn])

new_born.shape

(2561922, 61)

In [3]:
new_born.columns

Index(['ORIGEM', 'CODESTAB', 'CODMUNNASC', 'LOCNASC', 'IDADEMAE', 'ESTCIVMAE',
       'ESCMAE', 'CODOCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'CODMUNRES',
       'GESTACAO', 'GRAVIDEZ', 'PARTO', 'CONSULTAS', 'DTNASC', 'HORANASC',
       'SEXO', 'APGAR1', 'APGAR5', 'RACACOR', 'PESO', 'IDANOMAL', 'DTCADASTRO',
       'CODANOMAL', 'NUMEROLOTE', 'VERSAOSIST', 'DTRECEBIM', 'DIFDATA',
       'DTRECORIGA', 'NATURALMAE', 'CODMUNNATU', 'CODUFNATU', 'ESCMAE2010',
       'SERIESCMAE', 'DTNASCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR',
       'QTDPARTCES', 'IDADEPAI', 'DTULTMENST', 'SEMAGESTAC', 'TPMETESTIM',
       'CONSPRENAT', 'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO',
       'TPNASCASSI', 'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1',
       'STDNEPIDEM', 'STDNNOVA', 'CODPAISRES', 'TPROBSON', 'PARIDADE',
       'KOTELCHUCK', 'CONTADOR'],
      dtype='object')

https://pcdas.icict.fiocruz.br/conjunto-de-dados/sistema-de-informacao-sobre-nascidos-vivos/dicionario-de-variaveis/

In [4]:
# new_born = new_born[['DTNASC', 'HORANASC', 'IDADEMAE', 'CODMUNRES', 'PARTO', 'QTDFILMORT']]
new_born = new_born[['DTNASC', 'HORANASC', 'IDADEMAE', 'IDADEPAI', 'CODMUNRES', 'PARTO']]
new_born['IDADEMAE'] = new_born['IDADEMAE'].astype(float)

new_born = new_born[new_born['IDADEMAE'] < 14]

In [5]:
cities = pd.read_csv('src/data/CADMUN.csv', on_bad_lines = 'skip', encoding='latin-1', sep=';')

cities.shape

(5652, 28)

In [6]:
cities.columns

Index(['MUNCOD', 'MUNCODDV', 'SITUACAO', 'MUNSINP', 'MUNSIAFI', 'MUNNOME',
       'MUNNOMEX', 'OBSERV', 'MUNSINON', 'MUNSINONDV', 'AMAZONIA', 'FRONTEIRA',
       'CAPITAL', 'UFCOD', 'MESOCOD', 'MICROCOD', 'MSAUDCOD', 'RSAUDCOD',
       'CSAUDCOD', 'RMETRCOD', 'AGLCOD', 'ANOINST', 'ANOEXT', 'SUCESSOR',
       'LATITUDE', 'LONGITUDE', 'ALTITUDE', 'AREA'],
      dtype='object')

In [7]:
cities['MUNCOD'] = cities['MUNCOD'].astype(str)

cities = cities[['MUNCOD', 'MUNNOME', 'UFCOD', 'LATITUDE', 'LONGITUDE']]

In [8]:
new_born = new_born.merge(cities, how='left', left_on=['CODMUNRES'], right_on=['MUNCOD'])

new_born

,DTNASC,HORANASC,IDADEMAE,IDADEPAI,CODMUNRES,PARTO,MUNCOD,MUNNOME,UFCOD,LATITUDE,LONGITUDE
0,11052022,2125,13.0,NaN,500440,1,500440,Inocência,50.0,-19.726,-51.930
1,02012022,1610,13.0,18,500770,1,500770,Sete Quedas,50.0,-23.970,-55.036
2,01012022,1500,13.0,19,500370,1,500370,Dourados,50.0,-22.221,-54.806
3,18012022,0245,13.0,15,500540,1,500540,Maracaju,50.0,-21.614,-55.168
4,14012022,1234,13.0,20,500795,1,500795,Tacuru,50.0,-23.633,-55.016
...,...,...,...,...,...,...,...,...,...,...,...
3580,20122022,0140,13.0,NaN,330455,1,330455,Rio de Janeiro,33.0,-22.903,-43.208
3581,31122022,0804,13.0,NaN,330455,1,330455,Rio de Janeiro,33.0,-22.903,-43.208
3582,08122022,0727,13.0,NaN,330455,1,330455,Rio de Janeiro,33.0,-22.903,-43.208
3583,03112022,0201,13.0,NaN,330455,1,330455,Rio de Janeiro,33.0,-22.903,-43.208


In [9]:
new_born.groupby(['IDADEPAI'], as_index=False, dropna=False).agg({'DTNASC': 'count'})

,IDADEPAI,DTNASC
0,12,1
1,13,5
2,14,5
3,15,29
4,16,47
5,17,81
6,18,84
7,19,59
8,20,42
9,21,23


In [10]:
my_map = folium.Map(
                    location = [-11.733350280370129, -49.05221944693883], # Centralizar no Brasil
                    zoom_start = 5,
                    tiles = 'OpenStreetMap'
                    )

lat = new_born[new_born.LATITUDE.notnull()].LATITUDE.tolist()
long = new_born[new_born.LONGITUDE.notnull()].LONGITUDE.tolist()

locations = list(zip(lat, long))

MarkerCluster(locations=locations, 
                    # popups=locations,
                    ).add_to(my_map)

In [11]:
my_map

In [12]:
my_map = folium.Map(
                    location = [-11.733350280370129, -49.05221944693883], # Centralizar no Brasil
                    zoom_start = 5,
                    tiles = 'OpenStreetMap'
                    )

HeatMap(data=locations, radius=14).add_to(my_map)

folium.plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(my_map)

In [13]:
my_map

In [14]:
my_map = folium.Map(
                    location = [-11.733350280370129, -49.05221944693883], # Centralizar no Brasil
                    zoom_start = 5
                    )

HeatMap(data=locations, radius=14, blur=15, min_opacity=.4).add_to(my_map)

#Add the Stadia Maps Stamen Toner provider details via xyzservices
tile_provider = xyz.Stadia.StamenToner

#Update the URL to include the API key placeholder
tile_provider["url"] = tile_provider["url"] + "?api_key=038053bb-91f7-4660-897e-d8b6ae6e9191"

#Create the folium TileLayer, specifying the API key
folium.TileLayer(
    tiles=tile_provider.build_url(api_key='038053bb-91f7-4660-897e-d8b6ae6e9191'),
    attr=tile_provider.attribution,
    name=tile_provider.name,
    max_zoom=tile_provider.max_zoom,
    detect_retina=True
).add_to(my_map)

# folium.LayerControl().add_to(my_map)

In [15]:
# HeatMap(data=locations, radius=14).add_to(my_map)

my_map

In [16]:
my_map.save("results/too_young_map.html")